In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
#import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("poster")

In [2]:
import requests
import time
import json

In [101]:

docs=[]
ns = 1 #number of pages

#get first 100 pages of json dictionaries
for i in range(1,101): 
    url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page=%d&begin_date=20150101&end_date=20150114&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790" % i
    page = requests.get(url)
    pagedocs = page.json()['response']['docs']
    time.sleep(1)
    
    #append each separate dictionary from a page to docs
    for j in range(0,len(pagedocs)):
        docs.append(pagedocs[j])

In [102]:
fd=open("data/nytdocs-19.json","w")
json.dump(docs, fd)
fd.close()

In [117]:
docsinfo = []
for d in docs:
    obs = {}
    obs['id'] = d['_id']
    obs['type'] = d['type_of_material']
    obs['doctype'] = d['document_type']
    obs['date'] = d['pub_date']
    obs['news_desk'] = d['news_desk']
    obs['section'] = d['section_name']
    obs['subsection'] = d['subsection_name']
    obs['abstract'] = d['abstract']
    obs['paragraph'] = d['lead_paragraph']
    
    if d['headline'].get('main') is not None:
        obs['headline'] = d['headline']['main']
    else:
        obs['headline'] = d['headline']['name']
    
    if obs['date'] is not None:
        obs['date'] = obs['date'][0:10]
    
    #Take out abstracts and lead paragraphs with none to join text
    if obs['abstract'] is None:
        a = ' '
    else: 
        a = obs['abstract']
    if obs['paragraph'] == 'TK TK TK' or obs['paragraph'] is None:
        p = ' '
    else:
        p = obs['paragraph']
    
    text = [obs['headline'], p, a]
    obs['text'] = " ".join(text)
    
    docsinfo.append(obs)

In [118]:
docsdf = pd.DataFrame(docsinfo)
docsdf.head(2)

,abstract,date,doctype,headline,id,news_desk,paragraph,section,subsection,text,type
0,Dallas Journal; last person in Dallas being mo...,2014-11-08,article,Dallas Closes the Door on Its Ebola Scare,545ce19d79881039320ed5e9,National,The last person in the region being monitored ...,U.S.,None,Dallas Closes the Door on Its Ebola Scare The ...,News
1,When President Obama meets with the congressio...,2014-11-06,blogpost,Details on Obama's Ebola Funding Request,545bedac79881039320ecf87,None,None,None,None,Details on Obama's Ebola Funding Request Whe...,Blog


In [119]:
docsdf.shape

(646, 11)

In [120]:
docsdf['date'] = pd.to_datetime(docsdf['date'])

In [121]:
docsdf.to_csv("data/docsdf-50.csv")

## Automated code

In [ ]:
%%time
count = 92
months = ['01', '01', '01', '02', '02', '02', '03', '03', '03', '04', '04', '04', '05', '05', '05', '06', '06', '06', '07', '07', '07', '08', '08', '08', '09', '09', '09', '10', '10', '10', '11', '11', '11', '12', '12', '12']
startdays = ['01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21']
enddays = ['10', '20', '31', '10', '20', '27', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '30', '10', '20', '31']

pcount = 0

for d in range(36):
    sdate = '2012' + month[d] + startday[d]
    edate = '2012' + month[d] +  endday[d]

    docs=[]
        
    #get first 100 pages of json dictionaries
    for i in range(1,101): 
        url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page={}&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(i, sdate, edate)
        page = requests.get(url)
        pagedocs = page.json()['response']['docs']
        time.sleep(1)
        
        pcount += len(pagedocs)
        #append each separate dictionary from a page to docs
        for j in range(0,len(pagedocs)):
            docs.append(pagedocs[j])

        
    docsinfo = []
    for d in docs:
        obs = {}
        obs['id'] = d['_id']
        obs['type'] = d['type_of_material']
        obs['doctype'] = d['document_type']
        obs['date'] = d['pub_date']
        obs['news_desk'] = d['news_desk']
        obs['section'] = d['section_name']
        obs['subsection'] = d['subsection_name']
        obs['abstract'] = d['abstract']
        obs['paragraph'] = d['lead_paragraph']
        
        if d['headline'].get('main') is not None:
            obs['headline'] = d['headline']['main']
        else:
            obs['headline'] = d['headline']['name']
    
        if obs['date'] is not None:
            obs['date'] = obs['date'][0:10]
    
        #Take out abstracts and lead paragraphs with none to join text
        if obs['abstract'] is None:
            a = ' '
        else: 
            a = obs['abstract']
        if obs['paragraph'] == 'TK TK TK' or obs['paragraph'] is None:
            p = ' '
        else:
            p = obs['paragraph']
    
        text = [obs['headline'], p, a]
        obs['text'] = " ".join(text)
    
        docsinfo.append(obs)
    
    docsdf = pd.DataFrame(docsinfo)
    docsdf['date'] = pd.to_datetime(docsdf['date'])
    docsdf.to_csv("data/docsdf-{}.csv".format(count)) 

    print(count, pcount)
    count += 1 
    

92 363
93

## Other stuff

In [31]:
docsdf.dtypes

abstract              object
date          datetime64[ns]
doctype               object
headline              object
id                    object
news_desk             object
paragraph             object
section               object
subsection            object
text                  object
type                  object
dtype: object

In [ ]:
#with open("nytdocs100.json") as json_file:
#    docs = json.load(json_file)

Make date column a date type

http://stackoverflow.com/questions/9504356/convert-string-into-date-type-on-python

http://stackoverflow.com/questions/16852911/how-do-i-convert-dates-in-a-pandas-data-frame-to-a-date-data-type  ***

In [23]:
docsdf.to_csv("docsdf100-2.csv")  #, encoding = 'utf-8')

In [9]:
docsdf = pd.read_csv("docsdf100-2.csv", encoding = "ISO-8859-1")
docsdf.head()

#http://stackoverflow.com/questions/18171739/unicodedecodeerror-win-reading-csv-file-in-pandas-with-python
#http://stackoverflow.com/questions/24616678/unicodedecodeerror-in-python-when-reading-a-file-how-to-ignore-the-error-and-ju

,Unnamed: 0,abstract,date,doctype,headline,id,news_desk,paragraph,section,subsection,text,type
0,0,NaN,2015-11-03,article,"Know Your Risks, but Meat Still Isnt the Enemy",56374e0a798810408f299696,Upshot,The link between processed meat and cancer is ...,The Upshot,NaN,"Know Your Risks, but Meat Still Isnt the Enem...",News
1,1,Study in journal The Lancet concludes Chinese ...,2015-10-13,article,Study Shows Spread of Cigarettes in China,561bc3a67988101d11e0da9e,Science,Chinese men smoke one-third of all the worlds...,Health,NaN,Study Shows Spread of Cigarettes in China Chin...,News
2,2,Food and Drug Administration approves Narcan n...,2015-11-19,article,F.D.A. Approves a Nasal Spray to Combat Opioid...,564d433579881039a958bb07,National,A nasal spray designed to reverse opioid overd...,U.S.,NaN,F.D.A. Approves a Nasal Spray to Combat Opioid...,Brief
3,3,NaN,2015-11-17,article,How to Decrease Prices for an Expensive Class ...,5649c243798810095961385d,Upshot,Two professors suggest requiring biologic manu...,The Upshot,NaN,How to Decrease Prices for an Expensive Class ...,News
4,4,NaN,2015-10-07,article,Ragout With Farfalle Gets a Touch of Green,56143f92798810321b0fac91,Dining,The sweet taste and texture of broccoli leaves...,Food,NaN,Ragout With Farfalle Gets a Touch of Green The...,News


In [115]:
#solving keyerror issue for headline

ex = {'main': 'words', 'kicker': 'stuff'}
if ex.get('name') is None:
    print('ok')

#http://stackoverflow.com/questions/6130768/return-none-if-dictionary-key-is-not-available

ok
